In [2]:
pip install pymongo

In [6]:
import pymongo

In [7]:
from pymongo import MongoClient

In [8]:
connection = MongoClient('localhost',27017)
db = connection.NewsSeeker

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [11]:
data = db.MergedNews

In [12]:
df_merge = pd.DataFrame(list(data.find()))
df_merge

,_id,,title,text,subject,date,class
0,6059bbcc1a872bbab028f67d,0,Donald Trump Sends Out Embarrassing New Year’s...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,6059bbcc1a872bbab028f67e,1,Drunk Bragging Trump Staffer Started Russian C...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,6059bbcc1a872bbab028f67f,2,Sheriff David Clarke Becomes An Internet Joke ...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,6059bbcc1a872bbab028f680,3,Trump Is So Obsessed He Even Has Obama’s Name ...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,6059bbcc1a872bbab028f681,4,Pope Francis Just Called Out Donald Trump Duri...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...,...,...
44873,6059bbcf1a872bbab029a5c6,21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
44874,6059bbcf1a872bbab029a5c7,21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
44875,6059bbcf1a872bbab029a5c8,21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
44876,6059bbcf1a872bbab029a5c9,21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1


In [43]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [44]:
df.isnull().sum()

_id      0
         0
text     0
class    0
dtype: int64

In [45]:
df = df.sample(frac = 1)

In [46]:
df.head(10)

,_id,,text,class
12008,6059bbcd1a872bbab0292565,12008,"Megyn Kelly, who arrived at Fox News 12 years ...",0
37425,6059bbcf1a872bbab02988ae,13954,PARIS (Reuters) - The head of a Saudi-based or...,1
360,6059bbcc1a872bbab028f7e5,360,The Republican Party still hasn t learned that...,0
32273,6059bbcf1a872bbab029748e,8802,WASHINGTON (Reuters) - Democrat Baron Hill on ...,1
29526,6059bbce1a872bbab02969d3,6055,WASHINGTON/CHICAGO (Reuters) - U.S. President ...,1
32780,6059bbcf1a872bbab0297689,9309,SINGAPORE (Reuters) - The United States expect...,1
35591,6059bbcf1a872bbab0298184,12120,VIENNA (Reuters) - Austria s center-right Peop...,1
6512,6059bbcc1a872bbab0290fed,6512,Never underestimate the impact a single person...,0
33399,6059bbcf1a872bbab02978f4,9928,(Reuters) - Former New York Governor George Pa...,1
35012,6059bbcf1a872bbab0297f41,11541,SEOUL (Reuters) - South Korean President Moon ...,1


In [47]:
def drop_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [48]:
df["text"] = df["text"].apply(drop_text)

In [26]:
df.head(10)

,_id,,text,class
40054,6059bbcf1a872bbab02992f3,16583,reuters peruvian villagers suing miner gle...,1
4021,6059bbcc1a872bbab0290632,4021,the domestic terrorists who staged an armed ta...,0
2641,6059bbcc1a872bbab02900ce,2641,according to gop congressman sean duffy we ne...,0
19320,6059bbcd1a872bbab02941f5,19320,president elect donald trump came to the defen...,0
35972,6059bbcf1a872bbab0298301,12501,the hague reuters the daughter of an irani...,1
15027,6059bbcd1a872bbab0293130,15027,these leftist professors aren t just donating ...,0
13881,6059bbcd1a872bbab0292cb6,13881,two white guys living in a state where of it...,0
18861,6059bbcd1a872bbab029402a,18861,how long before the media will stop using two ...,0
29335,6059bbce1a872bbab0296914,5864,geneva reuters u s president donald trump...,1
21035,6059bbcd1a872bbab02948a8,21035,it s hard to tell if the men assaulting this f...,0


In [27]:
x = df["text"]
y = df["class"]

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [31]:
from sklearn.tree import DecisionTreeClassifier

In [32]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [33]:
pred_dt = DT.predict(xv_test)

In [34]:
DT.score(xv_test, y_test)

0.9954545454545455

In [35]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5906
           1       1.00      1.00      1.00      5314

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [36]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(drop_text) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_DT = DT.predict(new_xv_test)

    return print("\n \nDT Prediction: {} ".format(output_lable(pred_DT[0])))

In [37]:
news = str(input())
manual_testing(news)

GENEVA (Reuters) - North Korea and the United States accused each other on Tuesday of posing a nuclear threat, with Pyongyang s envoy declaring it would never put its atomic arsenal up for negotiation. The debate at the United Nations began when the U.S. envoy said President Donald Trump s top priority was to protect the United States and its allies against the  growing threat  from North Korea. To do so, he said, the country was ready to use  the full range of capabilities at our disposal . U.S. Ambassador Robert Wood told the Conference on Disarmament that the  path to dialogue still remains an option  for Pyongyang, but that Washington was  undeterred in defending against the threat North Korea poses . Fears have grown over North Korea s development of missiles and nuclear weapons since Pyongyang test-launched intercontinental ballistic missiles (ICBMs) in July. Those fears worsened after Trump warned that North Korea would face  fire and fury  if it threatened the United States. Hi